# FMR Analysis automation:



### 1. Configuration: 
*path:*
- `data_folder`: location of the measured `.txt/.csv`.  
- `results_root`: output root.  

*reading:*
- Read all `.txt/.csv` files inside `data_folder`.
- Parse metadata from filenames  ( sample name, frequency, db value)
- AZ5_f5GHz_m20dB_1.txt, AZ5_f5GHz_m20dB_2.csv files name are like that

*column setup:* 
- Validate required columns, map first column to current, and second column to dp\dH.  
- Normalize units (**f→GHz, I→A, H→T**).  
- insert a new column between I and the other, and compute **\(H = K*I + H_0\)**.  

*pre-processed files:* 
- save processed data, as .txt /csv in a results root, under same name,  


*parameter:*
-  Linear Relationship:\( H = K*I + H_0 \) with `k = 64.404`, `H_0 = 7.918`.    
- for debugging cause giving the needed `f` frequency to work on is handfull.



 

> Tip: we'll see if we could load a configure `config.json`\ or `.yaml` as default configuration. 

In [ ]:
# loading_setup(data_path, results_root, sample , f , db ); 
# it's single spectrum loading function, 
    # data_path = path to the data file,
    # results_root: root path to results folder,
    # sample: sample name, so it can path to subfolder inside results_root,
    # f: frequency,
    # db: decibel value.
# a default value could be set : path, sample, f = 3 (GHz), db = -10 (dB)
# AZ5_f5GHz_m20dB_1.txt, AZ5_f5GHz_m20dB_2.csv files name are like that
# head data/AZ5_f5GHz_m20dB_1.txt example: 
# 1.500000        ,       -0.009819
#         1.500000        ,       -0.026088
#         1.500000        ,       -0.024799
#         1.500000        ,       -0.072158
#         1.500000        ,       -0.062976
#         1.498000        ,       -0.096964
#         1.498000        ,       -0.045257
#         1.498000        ,       -0.034786
#         1.498000        ,       -0.099220
#         1.496000        ,       -0.204085
# head data/AZ5_f5GHz_m20dB_2.csv example:
# "1.500000	",#VALUE!,"	-0.029310	"
# "	1.500000	",,"	-0.084078	"
# "	1.500000	",,"	-0.108885	"
# "	1.500000	",,"	-0.089232	"
# "	1.500000	",,"	-0.062493	"
# "	1.498000	",,"	-0.059754	"
# "	1.498000	",,"	-0.094065	"
# "	1.498000	",,"	-0.049767	"

In [ ]:
# import sys
# sys.path.append('../scripts')  # from notebooks/ directory, go up one level to scripts

# from notebook_utils_Version2 import loading_setup

# data_path = "../data/AZ5_f5GHz_m20dB_1.txt"
# results_root = "../results_roots"
# sample = "AZ5"
# f = 5
# db = -20

# loading_setup(data_path, results_root, sample, f, db)

: 

### 2. ploting data: 
- ploting signal vs H, 
    ( the plot is just for visualization of raw data, before any fitting model )
    ( to make sure plot between column B and C ( i.e. H and dP/dH))

In [ ]:
# plot_spectrum(sample, f, db); # the plotting function

### 2. Single spectrum fits 
- a general function that take needed fitting function as parameter
- define fitting model: 
    - By lorentzain
    - By double lorentzian
    - or other needed to be defined
- the fit should be done on the plotted data, i.e. processed file in results
- Auto-initial guess with peak detection
- Fit with baseline correction
- Output per file: ( may be of csv, or txt), 
    - Output per file: H_res, ΔH_pp, ΔH_½, amplitude, fit_quality, uncertainties
    - Quality flags: SNR, residual analysis, single peak validation

In [ ]:
# fit_spectrum(results_root, fitting_model); # the fitting function
                                            # we will create an alias for different fitting model
                                            # fitting_model: lorentzian, double_lorentzian, asymmetric_lorentzian, ...
                                            # as always defaults parameter are registered based on parsed data in file names, but they could be changed in user request

### 3. automation process 
- Iterate through all files automatically, 
    - by same procedure as above
- output of fitting process over all files is one table with one line for each file, each output value should have it's column, 
- with saved plots. 

### 4. merging plots.
- for each processed data, the plots will be saved a with header (title)  sample name, frequency.
- at the end the plots will be merged into pptx document each in one slide. 



In [ ]:
# automation_analysis(results_root, fitting_model);
# # the main script to call all functions above 
# # this will be a function that will loop over all data files in the results_root folder
# # and call the loading, plotting, and fitting functions automatically
# # parameters could be set as default or user defined,
# # ouptput per sample: named: sample_name_analysis.xlsx in excel file format, 
# # excel file contain exactly in this order: f (GHz), H_res , error( on H_res), Delta_H, error (on Delta_H), R2, Adj_R2, RMSE, SSE, MSE, fitting_model

# # function will be created as modules in a package, so no need to define main function here, or write it's script here.
# # here we just define the function calls and their parameters.

# merge_plots_to_pptx(results_root, pptx_name);
# # this function will merge all saved plots into one pptx file,
# # pptx_name: sample_name_frequency.pptx
# # results_root: the folder that contain all data files,  # noting: load processed files


### 5. Batch Kittel Fitting
- Kittel analysis consist of plotting  frequency in terms of H_res + error 
- Fitting to kittel ( a defined fonction) `f = γ/2π * √{H(H + M)} `
- -> extract from fitting: `γ/2π` and `M_eff` with confidence intervals 
- output per sample: name:  damping_analysis.xlsx , with fitting parameter 
 

In [ ]:
# Kittel_fitting(results_root, kittel_output_name);
# we may create a function that will do the kittel fitting automatically for all samples
# or use the same fit_spectrum function with a different fitting model

### 6. Batch Damping Extraction
from the same excell file; 
- plot `ΔH vs f` 
    - fitting model here is: $ ΔH = \frac{2α}{\gamma} f + ΔH_0 $
    - fitted parmater: slope $ \frac{2α}{γ} $ and y-intercept: `\Delta{H_0}` should be saved into same file damping_analysis, 

- damping is then calculated from slope: ` α = γ\2 * slope `
- Output per sample: α, ΔH₀ with confidence intervals

In [ ]:
# damping_fitting(results_root, damping_output_name);
# we may create a function that will do the kittel fitting automatically for all samples
# or use the same fit_spectrum function with a different fitting model

### 7. Quality assurance dashboard 
**What to check:**
- needed results to check and log

In [ ]:
# ouptput repo should contain: 
#

### 8. Results Export & Reporting
- results table: all samples with parameters and uncertainties
- Exporting figures in PNG/SVG format
- CSV/Excel export of all analysis results
- Summary report with QA status and key findings`

### 9. One‑click pipeline (default settings)
Runs the full chain with sensible defaults. Use this when you don’t need to tweak anything.

**Run (one call):**